In [1]:
import numpy as np
import pandas as pd

In [15]:
!bwa index data/MG1655-K12.first400K.fasta
!bwa mem data/MG1655-K12.first400K.fasta data/ecoli_400K_err_1.fastq data/ecoli_400K_err_2.fastq > ecoli_400.sam

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.08 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.04 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index data/MG1655-K12.first400K.fasta
[main] Real time: 0.141 sec; CPU: 0.125 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 100000 sequences (10000000 bp)...
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48301, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.27, 10.16)
[M::mem_pestat] low and high boundaries for proper pair

[M::mem_process_seqs] Processed 100000 reads in 3.533 CPU sec, 3.393 real sec
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48231, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.27, 10.13)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 100000 reads in 3.524 CPU sec, 3.394 real sec
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48187, 0, 0)
[M::mem_pestat] skip orientation FF as there are n

[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48361, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.23, 10.15)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 100000 reads in 3.227 CPU sec, 3.095 real sec
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48350, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orienta

In [32]:
def cnt_err_del(sam, fasta, n_as_error=False, start_positions=None):

    with open(fasta) as file:
        reference = ''.join([line.strip() for line in file if not line.startswith('>')])
    ref_length = len(reference)
    
    
    n_errors, n_deletions = {}, {}
    correct_positions = 0
    
    for num, line in enumerate(open(sam)):
        if line.startswith('@'):
            continue
            
        data = line.strip().split()
        if data[9] == '*':
            continue
        start = int(data[3])
        if not start:
            continue
            
        read_name = data[0]
        read_type = 'forward' if int(data[8]) > 0 else 'backward'
        
        if start_positions and (read_name, read_type) in start_positions:
            start_ecoli_pos = start_positions[(read_name, read_type)]
            if np.abs(start_ecoli_pos - start) > 100:
                continue
                
            start = start_ecoli_pos
            
        if not start:
            continue

        errors = {}
        deletions = {}
        read = list(data[9])
        read_length = len(read)
        reference_region = reference[start - 1 : min(start - 1 + read_length, ref_length)]
        
        for pos, r in enumerate(reference_region):
            if r != read[pos]:
                if read[pos] == 'N' and not n_as_error:
                    deletions[start + pos] = (r, read[pos])
                else:
                    errors[start + pos] = (r, read[pos])
            else:
                correct_positions += 1
                
        if len(errors) > 0:
            n_errors[(read_name, read_type)] = errors
            
        if len(deletions) > 0:
            n_deletions[(read_name, read_type)] = deletions
    return n_errors, n_deletions, correct_positions


def read_names(sam):
    names = set()
    for num, line in enumerate(open(sam)):
        if line.startswith('@'):
            continue
            
        data = line.strip().split()
        if data[2] == '*' or data[5] == '*':
            continue
            
        start = int(data[3])
        if not start:
            continue
            
        read_name = data[0]
        read_type = 'forward' if int(data[8]) > 0 else 'backward'
        names.add((read_name, read_type))
    return names


def deleted(total_reads, sam):
    deleted_reads = total_reads.copy()
    for num, line in enumerate(open(sam)):
        if line.startswith('@'):
            continue
            
        data = line.strip().split()
        if data[2] == '*' or data[5] == '*':
            continue
            
        start = int(data[3])
        if not start:
            continue
            
        read_name = data[0]
        read_type = 'forward' if int(data[8]) > 0 else 'backward'
        if (read_name, read_type) in total_reads:
            deleted_reads.remove((read_name, read_type))
    return deleted_reads


def deleted_correct_positions(errors, deletions, deleted_reads, sam):
    n = 0
    for num, line in enumerate(open(sam)):
        if line.startswith('@'):
            continue
            
        data = line.strip().split()
        if data[2] == '*' or data[5] == '*':
            continue
            
        start = int(data[3])
        if not start:
            continue
            
        read_name = data[0]
        read_type = 'forward' if int(data[8]) > 0 else 'backward'
        read = list(data[9])
        
        if (read_name, read_type) in deleted_reads:
            read_length = len(read)
            if (read_name, read_type) not in errors:
                n += read_length
                continue
                
            errors_n = len(errors[(read_name, read_type)])
            n += (read_length - errors_n)
            
        elif (read_name, read_type) in deletions:
            errors_positions = errors[(read_name, read_type)] if (read_name, read_type) in errors else {}
            for pos in deletions[(read_name, read_type)]:
                if pos not in errors_positions:
                    n += 1
    return n

# Trimmomatic

In [7]:
!trimmomatic PE data/ecoli_400K_err_1.fastq data/ecoli_400K_err_2.fastq \
trim1_paired.fq trim1_unpaired.fq trim2_paired.fq trim2_unpaired.fq \
LEADING:5 TRAILING:5 SLIDINGWINDOW:4:15 MINLEN:35

TrimmomaticPE: Started with arguments:
 data/ecoli_400K_err_1.fastq data/ecoli_400K_err_2.fastq trim1_paired.fq trim1_unpaired.fq trim2_paired.fq trim2_unpaired.fq LEADING:5 TRAILING:5 SLIDINGWINDOW:4:15 MINLEN:35
Multiple cores found: Using 4 threads
Quality encoding detected as phred33
Input Read Pairs: 1381602 Both Surviving: 1314087 (95,11%) Forward Only Surviving: 36625 (2,65%) Reverse Only Surviving: 26152 (1,89%) Dropped: 4738 (0,34%)
TrimmomaticPE: Completed successfully


In [9]:
!bwa mem data/MG1655-K12.first400K.fasta trim1_paired.fq trim2_paired.fq > trim_paired.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 109032 sequences (10000046 bp)...
[M::process] read 109552 sequences (10000124 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 52822, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.25, 10.17)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 109032 reads in 3.460 CPU sec, 3.388 real sec
[M::process] read 109748 sequences (10000159 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 53112, 0, 0)
[M::mem_pestat] skip orientat

[M::process] read 109106 sequences (10000148 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 54275, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.36, 10.13)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 112038 reads in 3.216 CPU sec, 3.061 real sec
[M::process] read 108334 sequences (10000075 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 52845, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orienta

[M::mem_process_seqs] Processed 105670 reads in 2.934 CPU sec, 2.767 real sec
[M::process] read 15636 sequences (1488812 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 51099, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.26, 10.13)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 105478 reads in 2.901 CPU sec, 2.790 real sec
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 7576, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size d

In [16]:
raw_errors, raw_deletions, raw_correct_positions = cnt_err_del('ecoli_400.sam', 'data/MG1655-K12.first400K.fasta')

In [18]:
trim_errors, trim_deletions, trim_correct_positions = cnt_err_del('trim_paired.sam', 
                                                                  'data/MG1655-K12.first400K.fasta')

In [27]:
total_raw_errors = 0
for positions in raw_errors.values():
    total_raw_errors += len(positions)
total_trim_errors = 0
for positions in trim_errors.values():
    total_trim_errors += len(positions)
    
result = pd.DataFrame(columns=['Error in corrected reads', 'Correct base in corrected reads', 
                               'Base is absent in corrected reads'], 
                      index=['Error in raw data', 'Correct base in raw data'])

result.loc['Error in raw data', 'Error in corrected reads'] = f'{total_trim_errors} ({total_trim_errors / total_raw_errors * 100:.3f}%)'

result.loc['Error in raw data', 'Correct base in corrected reads'] = f'0'

result.loc['Error in raw data', 'Base is absent in corrected reads'] = f'{total_raw_errors - total_trim_errors} ({(total_raw_errors - total_trim_errors) / total_raw_errors * 100:.3f}%)'

result.loc['Correct base in raw data', 'Error in corrected reads'] = f'0'

result.loc['Correct base in raw data', 'Correct base in corrected reads'] = f'{trim_correct_positions} ({trim_correct_positions / raw_correct_positions * 100:.3f}%)'

result.loc['Correct base in raw data', 'Base is absent in corrected reads'] = f'{raw_correct_positions - trim_correct_positions} ({(raw_correct_positions - trim_correct_positions) / raw_correct_positions * 100:.3f}%)'

In [28]:
result

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,1389165 (9.354%),0,13462255 (90.646%)
Correct base in raw data,0,239971114 (91.883%),21197833 (8.117%)


# Spades

In [47]:
def spades_table(errors1, errors2, deletions2, sam1, sam2, good1, good2):

    first_reads = read_names(sam1)
    second_deleted_reads = deleted(first_reads, sam2)
    
    total_reads = len(first_reads)
    deleted_reads = len(second_deleted_reads)
    n_errors = 0
    corrected_errors = 0
    deleted_errors = 0
    
    for (read, read_type), positions in errors1.items():
        n = len(positions)
        n_errors += n
        if (read, read_type) in second_deleted_reads:
            deleted_errors += n
            continue
            
        del_positions = deletions2[(read, read_type)] if (read, read_type) in deletions2 else {}
        err_positions = errors2[(read, read_type)] if (read, read_type) in errors2 else {}
        for pos in positions.keys():
            if pos in del_positions:
                deleted_errors += 1
            elif pos not in err_positions:
                corrected_errors += 1
            
    new_errors = 0
    for (read, read_type), positions in errors2.items():
        err_positions = errors1[(read, read_type)] if (read, read_type) in errors1 else {}
        for pos in positions.keys():
            if pos not in err_positions:
                new_errors += 1
    deleted_good_n = deleted_correct_positions(errors1, deletions2, second_deleted_reads, sam1)
    
    result = pd.DataFrame(columns=['Error in corrected reads', 'Correct base in corrected reads', 
                                   'Base is absent in corrected reads'], 
                          index=['Error in raw data', 'Correct base in raw data'])
    
    result.loc['Error in raw data', 'Error in corrected reads'] = f'{n_errors - deleted_errors - corrected_errors} ({(n_errors - deleted_errors - corrected_errors) / n_errors * 100:.3f}%)'
    
    result.loc['Error in raw data', 'Correct base in corrected reads'] = f'{corrected_errors} ({corrected_errors / n_errors * 100:.3f}%)'
    
    result.loc['Error in raw data', 'Base is absent in corrected reads'] = f'{deleted_errors} ({deleted_errors / n_errors * 100:.3f}%)'
    
    result.loc['Correct base in raw data', 'Error in corrected reads'] = f'{new_errors} ({new_errors / good1 * 100:.3f}%)'
    
    result.loc['Correct base in raw data', 'Correct base in corrected reads'] = f'{good1 - deleted_good_n - new_errors} ({(good1 - deleted_good_n - new_errors) / good1 * 100:.3f}%)'
    
    result.loc['Correct base in raw data', 'Base is absent in corrected reads'] = f'{deleted_good_n} ({deleted_good_n / good1 * 100:.3f}%)'
    return result

In [39]:
def read_start(sam):
    read2position = {}
    for num, line in enumerate(open(sam)):
        if line.startswith('@'):
            continue
            
        data = line.strip().split()
        if data[2] == '*' or data[5] == '*':
            continue
            
        start = int(data[3])
        if not start:
            continue
            
        read_name = data[0]
        read_type = 'forward' if int(data[8]) > 0 else 'backward'
        read2position[(read_name, read_type)] = start
    return read2position

In [29]:
!spades.py -1 data/ecoli_400K_err_1.fastq -2 data/ecoli_400K_err_2.fastq -o spades_ecoli --only-error-correction \
--threads 6



== Warning ==  No assembly mode was sepcified! If you intend to assemble high-coverage multi-cell/isolate data, use '--isolate' option.


Command line: /home/alex/anaconda3/bin/spades.py	-1	/home/alex/PycharmProjects/NGS/homework_4/data/ecoli_400K_err_1.fastq	-2	/home/alex/PycharmProjects/NGS/homework_4/data/ecoli_400K_err_2.fastq	-o	/home/alex/PycharmProjects/NGS/homework_4/spades_ecoli	--only-error-correction	--threads	6	

System information:
  SPAdes version: 3.14.0
  Python version: 3.7.4
  OS: Linux-5.4.0-33-generic-x86_64-with-debian-bullseye-sid

Output dir: /home/alex/PycharmProjects/NGS/homework_4/spades_ecoli
Mode: ONLY read error correction (without assembling)
Debug mode is turned OFF

Dataset parameters:
  Standard mode
  For multi-cell/isolate data we recommend to use '--isolate' option; for single-cell MDA data use '--sc'; for metagenomic data use '--meta'; for RNA-Seq use '--rna'.
  Reads:
    Library number: 1, library type: paired-end
      orientation: fr
      lef

  0:02:02.497   464M / 2G    INFO    General                 (main.cpp                  : 178)   Finished clustering.
  0:02:02.497   464M / 2G    INFO    General                 (main.cpp                  : 197)   Starting solid k-mers expansion in 6 threads.
  0:02:23.677   461M / 2G    INFO    General                 (main.cpp                  : 218)   Solid k-mers iteration 0 produced 45116 new k-mers.
  0:02:44.864   457M / 2G    INFO    General                 (main.cpp                  : 218)   Solid k-mers iteration 1 produced 1684 new k-mers.
  0:03:06.171   457M / 2G    INFO    General                 (main.cpp                  : 218)   Solid k-mers iteration 2 produced 63 new k-mers.
  0:03:32.174   457M / 2G    INFO    General                 (main.cpp                  : 218)   Solid k-mers iteration 3 produced 32 new k-mers.
  0:04:03.383   457M / 2G    INFO    General                 (main.cpp                  : 218)   Solid k-mers iteration 4 produced 0 new k-mers.
  0:0

In [31]:
!bwa mem data/MG1655-K12.first400K.fasta spades_ecoli/corrected/ecoli_400K_err_1.00.0_0.cor.fastq.gz \
spades_ecoli/corrected/ecoli_400K_err_2.00.0_0.cor.fastq.gz > spades_ecoli.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 100000 sequences (10000000 bp)...
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48352, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.27, 10.15)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 100000 reads in 3.047 CPU sec, 2.854 real sec
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48401, 0, 0)
[M::mem_pestat] skip orientat

[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48338, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.30, 10.14)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 100000 reads in 2.846 CPU sec, 2.540 real sec
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48261, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orienta

[M::mem_process_seqs] Processed 100000 reads in 2.761 CPU sec, 2.485 real sec
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48382, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (207, 214, 222)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (177, 252)
[M::mem_pestat] mean and std.dev: (214.31, 10.16)
[M::mem_pestat] low and high boundaries for proper pairs: (162, 267)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 100000 reads in 3.621 CPU sec, 3.291 real sec
[M::process] read 100000 sequences (10000000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 48355, 0, 0)
[M::mem_pestat] skip orientation FF as there are n

In [34]:
start_positions = read_start('ecoli_400.sam')

In [36]:
spades_errors, spades_deletions, spades_correct_positions = cnt_err_del('spades_ecoli.sam',
                                                                        'data/MG1655-K12.first400K.fasta',
                                                                        start_positions=start_positions)

In [48]:
result = spades_table(raw_errors, spades_errors, spades_deletions, 'ecoli_400.sam', 'spades_ecoli.sam', 
                      raw_correct_positions, spades_correct_positions)

In [49]:
result

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
Error in raw data,7605795 (51.213%),826684 (5.566%),6418941 (43.221%)
Correct base in raw data,26086 (0.010%),241818403 (92.591%),19324458 (7.399%)


In [ ]:
# вывод: кажется, что Trimmamatic справляется лучше, так как TP значительно выше, чем у Spades, TN мерно 
# одинаковы.